In [1]:
import pandas as pd, numpy as np, gensim, pickle, spacy, urllib.request
import nltk.tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class Corpus:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        i = 0
        fp = open(self.path)
        line = fp.readline()
        while line:
            yield line[:-1].split()
            line = fp.readline()
            i+=1
        fp.close()

In [0]:
data = pd.read_pickle("/content/drive/My Drive/Colab Notebooks/IFT6285/train_post_preprocessed2.pkl")

In [43]:
data

,text,label
0,"[long, time, ..., i, have, been, busy, with, w...",0
1,"[what, to, talk, about, ..., well, today, wa, ...",0
2,"[just, some, stuff, which, animal, is, your, s...",0
3,"[i, gots, a, job, now, ..., yesterday, i, star...",0
4,"[movie, night, !, i, really, do, hate, friday,...",0
...,...,...
512624,"[whenever, i, hear, my, band, warm, down, i, l...",1
512625,"[well, since, the, last, entry, me, and, my, g...",1
512626,"[sigh, well, tonight, me, and, my, girl, decid...",1
512627,"[if, anyone, is, awake, and, acutally, reading...",1


In [0]:
sentences = Corpus("data/preprocessed.txt")

In [46]:
%%time
word2vec = gensim.models.Word2Vec(min_count=10,
                                  sentences= data.text,
                                 size=16,
                                 workers=4)

CPU times: user 21min 25s, sys: 5.56 s, total: 21min 30s
Wall time: 11min 10s


In [0]:
word2vec.wv.vocab

In [0]:
fp = open("word2vec", "wb")
pickle.dump(word2vec, fp)
fp.close()

In [0]:
fp = open("/content/drive/My Drive/Colab Notebooks/IFT6285/word2vec_no_preprocess", "rb")
word2vec2 = pickle.load(fp)
fp.close()

In [0]:
url = 'http://www-labs.iro.umontreal.ca/~felipe/IFT6285-Automne2019/resources/tp2/train_posts.csv'
response = urllib.request.urlopen(url)
data = pd.read_csv(response, header=None)

In [0]:
word2vec = gensim.models.Word2Vec(min_count=10,
                                #   sentences= [s.split() for s in nltk.tokenize.sent_tokenize(data.values[0,0])],
                                  sentences = nltk.tokenize.sent_tokenize(data.values[0,0]),
                                 size=16,
                                 workers=4)

In [0]:
class Sentence:
    def __init__(self, posts):
        self.posts = posts
    
    def __iter__(self):
        for post in self.posts:
            for sent in nltk.tokenize.sent_tokenize(post):
                yield nltk.tokenize.word_tokenize(sent)

In [0]:
iter_sent = Sentence(data.values[:,0])

In [0]:
word2vec = gensim.models.Word2Vec(  min_count=10,
                                    sentences = iter_sent,
                                    size=16,
                                    workers=4)

In [0]:
fp = open("/content/drive/My Drive/Colab Notebooks/IFT6285/TP3/word2vec_no_preprocess_10", "wb")
pickle.dump(word2vec, fp)
fp.close()

In [39]:
word2vec.wv.most_similar("computer", topn=10)
# np.linalg.norm(word2vec.wv["salt"]  word2vec.wv["sugar"] )

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('laptop', 0.9458898305892944),
 ('pc', 0.942133903503418),
 ('notepad', 0.9138541221618652),
 ('mailbox', 0.902761697769165),
 ('workstation', 0.8988693356513977),
 ('harddrive', 0.8768801093101501),
 ('paper', 0.8752428293228149),
 ('standby', 0.873042643070221),
 ('net', 0.8727045655250549),
 ('routine', 0.8712579011917114)]

In [40]:
len(word2vec2.wv.vocab)

955252